In [1]:
import pandas as pd
import numpy as np

import geopandas as gpd
from shapely.geometry import Point
import rtree
import pickle

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

import secret_key

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [2]:
mta_df = pd.read_csv("data/mta_turnstile/turnstile_170107.txt")
mta_df = mta_df.append(pd.read_csv("data/mta_turnstile/turnstile_170114.txt"))
mta_df = mta_df.append(pd.read_csv("data/mta_turnstile/turnstile_170121.txt"))
mta_df = mta_df.append(pd.read_csv("data/mta_turnstile/turnstile_170128.txt"))

In [3]:
mta_df.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/31/2016,03:00:00,REGULAR,5991546,2028378
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/31/2016,07:00:00,REGULAR,5991565,2028389
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/31/2016,11:00:00,REGULAR,5991644,2028441
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/31/2016,15:00:00,REGULAR,5991971,2028502
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/31/2016,19:00:00,REGULAR,5992418,2028543


In [4]:
mta_df['DATE'] = pd.to_datetime(mta_df.DATE)

In [5]:
mta_df['month'] = mta_df.DATE.dt.month

In [6]:
mta_df['year'] = mta_df.DATE.dt.year

In [7]:
mta_df['day_of_week'] = mta_df.DATE.dt.weekday_name

In [8]:
mta_df.day_of_week.value_counts()

Tuesday      111687
Wednesday    111432
Friday       110914
Thursday     110841
Monday       110819
Sunday       110728
Saturday     110682
Name: day_of_week, dtype: int64

In [12]:
mta_df[mta_df.STATION == 'CENTRAL AV'].head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,month,year,day_of_week
38893,K017,R401,00-00-00,CENTRAL AV,M,BMT,2016-12-31,00:00:00,REGULAR,507074,1691710,12,2016,Saturday
38894,K017,R401,00-00-00,CENTRAL AV,M,BMT,2016-12-31,04:00:00,REGULAR,507081,1691803,12,2016,Saturday
38895,K017,R401,00-00-00,CENTRAL AV,M,BMT,2016-12-31,08:00:00,REGULAR,507099,1691827,12,2016,Saturday
38896,K017,R401,00-00-00,CENTRAL AV,M,BMT,2016-12-31,12:00:00,REGULAR,507175,1691884,12,2016,Saturday
38897,K017,R401,00-00-00,CENTRAL AV,M,BMT,2016-12-31,16:00:00,REGULAR,507256,1692008,12,2016,Saturday


In [23]:
import requests
import json
response = requests.get('https://maps.googleapis.com/maps/api/geocode/json?address=103+st+station+new+york+line+1&key='+secret_key.google_api_key)

In [24]:
#response.json()[['results']
test = json.loads(response.text)

In [27]:
test['results'][0]['geometry']['location']['lat'], test['results'][0]['geometry']['location']['lng']

(40.7902308, -73.9477564)

In [28]:
mta_loc = mta_df.groupby(['STATION', 'LINENAME', 'DIVISION'])['UNIT'].count().reset_index()

In [31]:
mta_loc = mta_loc.drop('UNIT', axis=1)

In [35]:
mta_loc.head()

,STATION,LINENAME,DIVISION
0,1 AV,L,BMT
1,103 ST,1,IRT
2,103 ST,6,IRT
3,103 ST,BC,IND
4,103 ST-CORONA,7,IRT


In [53]:
import time

In [69]:
for index, loc in mta_loc.iterrows():
    try:
        station = mta_loc.STATION[index]
        linename = mta_loc.LINENAME[index]
        division = mta_loc.DIVISION[index]

        search_query = "https://maps.googleapis.com/maps/api/geocode/json?address="
        for word in station.split(" "):
            search_query += word + '+'

        search_query += 'station+' + linename + '+new+york+'
        search_query += division + '&key=' + secret_key.google_api_key
#         print(search_query)
        response = requests.get(search_query)
        result = json.loads(response.text)

        mta_loc.loc[[index], 'lat'] = result['results'][0]['geometry']['location']['lat']
        mta_loc.loc[[index], 'long'] = result['results'][0]['geometry']['location']['lng']
    except:
        print(mta_loc.STATION[index])
    time.sleep(1)
        
    

111 ST
CENTRAL AV
HEWES ST


In [103]:
mta_loc[mta_loc.LINENAME == 'NQR456W']

,STATION,LINENAME,DIVISION,lat,long
124,59 ST,NQR456W,BMT,40.641568,-74.017678


In [97]:
mta_loc['lat'][328] = 40.706693

/home/iniyavan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [95]:
mta_loc['long'][328] = -73.952892

/home/iniyavan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [100]:
mta_loc['long'][244]

-73.927481

In [102]:
mta_loc.head()

,STATION,LINENAME,DIVISION,lat,long
0,1 AV,L,BMT,40.763368,-73.959240
1,103 ST,1,IRT,40.773697,-73.982229
2,103 ST,6,IRT,40.790231,-73.947756
3,103 ST,BC,IND,40.796289,-73.961658
4,103 ST-CORONA,7,IRT,40.749780,-73.862646


In [104]:
mta_df.shape

(777103, 14)

In [106]:
test = pd.merge(mta_df, mta_loc, on=['STATION', 'LINENAME', 'DIVISION'])

In [107]:
test.shape

(777103, 16)

In [108]:
test.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,month,year,day_of_week,lat,long
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,2016-12-31,03:00:00,REGULAR,5991546,2028378,12,2016,Saturday,40.641568,-74.017678
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,2016-12-31,07:00:00,REGULAR,5991565,2028389,12,2016,Saturday,40.641568,-74.017678
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,2016-12-31,11:00:00,REGULAR,5991644,2028441,12,2016,Saturday,40.641568,-74.017678
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,2016-12-31,15:00:00,REGULAR,5991971,2028502,12,2016,Saturday,40.641568,-74.017678
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,2016-12-31,19:00:00,REGULAR,5992418,2028543,12,2016,Saturday,40.641568,-74.017678


In [137]:
test['zone'] = test['STATION'] + " " + test['LINENAME'] + " " + test['DIVISION'] 

In [138]:
test.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,month,year,day_of_week,lat,long,zone
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,2016-12-31,03:00:00,REGULAR,5991546,2028378,12,2016,Saturday,40.641568,-74.017678,59 ST NQR456W BMT
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,2016-12-31,07:00:00,REGULAR,5991565,2028389,12,2016,Saturday,40.641568,-74.017678,59 ST NQR456W BMT
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,2016-12-31,11:00:00,REGULAR,5991644,2028441,12,2016,Saturday,40.641568,-74.017678,59 ST NQR456W BMT
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,2016-12-31,15:00:00,REGULAR,5991971,2028502,12,2016,Saturday,40.641568,-74.017678,59 ST NQR456W BMT
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,2016-12-31,19:00:00,REGULAR,5992418,2028543,12,2016,Saturday,40.641568,-74.017678,59 ST NQR456W BMT


In [111]:
bike_df = pd.read_csv("data/201701-citibike-tripdata.csv.zip")

In [113]:
bike_df['Start Time'] = pd.to_datetime(bike_df['Start Time'])

In [114]:
bike_df['Stop Time'] = pd.to_datetime(bike_df['Stop Time'])

In [115]:
bike_df['month'] = bike_df['Start Time'].dt.month

In [116]:
bike_df['year'] = bike_df['Start Time'].dt.year

In [117]:
bike_df['day_of_week'] = bike_df['Start Time'].dt.weekday_name

In [118]:
bike_df['time_of_day'] = bike_df['Start Time'].dt.hour

In [119]:
bike_df.head()

,Trip Duration,Start Time,Stop Time,Start Station ID,Start Station Name,Start Station Latitude,Start Station Longitude,End Station ID,End Station Name,End Station Latitude,End Station Longitude,Bike ID,User Type,Birth Year,Gender,month,year,day_of_week,time_of_day
0,680,2017-01-01 00:00:21,2017-01-01 00:11:41,3226,W 82 St & Central Park West,40.782750,-73.971370,3165,Central Park West & W 72 St,40.775794,-73.976206,25542,Subscriber,1965.0,2,1,2017,Sunday,0
1,1282,2017-01-01 00:00:45,2017-01-01 00:22:08,3263,Cooper Square & E 7 St,40.729236,-73.990868,498,Broadway & W 32 St,40.748549,-73.988084,21136,Subscriber,1987.0,2,1,2017,Sunday,0
2,648,2017-01-01 00:00:57,2017-01-01 00:11:46,3143,5 Ave & E 78 St,40.776829,-73.963888,3152,3 Ave & E 71 St,40.768737,-73.961199,18147,Customer,NaN,0,1,2017,Sunday,0
3,631,2017-01-01 00:01:10,2017-01-01 00:11:42,3143,5 Ave & E 78 St,40.776829,-73.963888,3152,3 Ave & E 71 St,40.768737,-73.961199,21211,Customer,NaN,0,1,2017,Sunday,0
4,621,2017-01-01 00:01:25,2017-01-01 00:11:47,3143,5 Ave & E 78 St,40.776829,-73.963888,3152,3 Ave & E 71 St,40.768737,-73.961199,26819,Customer,NaN,0,1,2017,Sunday,0


In [131]:
test[test.STATION == "103 ST"].head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,month,year,day_of_week,lat,long
181160,N037,R314,00-00-00,103 ST,BC,IND,2016-12-31,00:00:00,REGULAR,12341850,10396043,12,2016,Saturday,40.796289,-73.961658
181161,N037,R314,00-00-00,103 ST,BC,IND,2016-12-31,04:00:00,REGULAR,12341881,10396092,12,2016,Saturday,40.796289,-73.961658
181162,N037,R314,00-00-00,103 ST,BC,IND,2016-12-31,08:00:00,REGULAR,12341966,10396133,12,2016,Saturday,40.796289,-73.961658
181163,N037,R314,00-00-00,103 ST,BC,IND,2016-12-31,12:00:00,REGULAR,12342281,10396230,12,2016,Saturday,40.796289,-73.961658
181164,N037,R314,00-00-00,103 ST,BC,IND,2016-12-31,16:00:00,REGULAR,12342633,10396455,12,2016,Saturday,40.796289,-73.961658


In [129]:
bike_df[bike_df['Start Station Longitude'] == -73.927481]

,Trip Duration,Start Time,Stop Time,Start Station ID,Start Station Name,Start Station Latitude,Start Station Longitude,End Station ID,End Station Name,End Station Latitude,End Station Longitude,Bike ID,User Type,Birth Year,Gender,month,year,day_of_week,time_of_day


In [142]:
mini_bike = bike_df.head(100)

In [144]:
mini_bike.head()

,Trip Duration,Start Time,Stop Time,Start Station ID,Start Station Name,Start Station Latitude,Start Station Longitude,End Station ID,End Station Name,End Station Latitude,End Station Longitude,Bike ID,User Type,Birth Year,Gender,month,year,day_of_week,time_of_day
0,680,2017-01-01 00:00:21,2017-01-01 00:11:41,3226,W 82 St & Central Park West,40.782750,-73.971370,3165,Central Park West & W 72 St,40.775794,-73.976206,25542,Subscriber,1965.0,2,1,2017,Sunday,0
1,1282,2017-01-01 00:00:45,2017-01-01 00:22:08,3263,Cooper Square & E 7 St,40.729236,-73.990868,498,Broadway & W 32 St,40.748549,-73.988084,21136,Subscriber,1987.0,2,1,2017,Sunday,0
2,648,2017-01-01 00:00:57,2017-01-01 00:11:46,3143,5 Ave & E 78 St,40.776829,-73.963888,3152,3 Ave & E 71 St,40.768737,-73.961199,18147,Customer,NaN,0,1,2017,Sunday,0
3,631,2017-01-01 00:01:10,2017-01-01 00:11:42,3143,5 Ave & E 78 St,40.776829,-73.963888,3152,3 Ave & E 71 St,40.768737,-73.961199,21211,Customer,NaN,0,1,2017,Sunday,0
4,621,2017-01-01 00:01:25,2017-01-01 00:11:47,3143,5 Ave & E 78 St,40.776829,-73.963888,3152,3 Ave & E 71 St,40.768737,-73.961199,26819,Customer,NaN,0,1,2017,Sunday,0


In [146]:
mta_loc.head()

,STATION,LINENAME,DIVISION,lat,long
0,1 AV,L,BMT,40.763368,-73.959240
1,103 ST,1,IRT,40.773697,-73.982229
2,103 ST,6,IRT,40.790231,-73.947756
3,103 ST,BC,IND,40.796289,-73.961658
4,103 ST-CORONA,7,IRT,40.749780,-73.862646


In [151]:
for index, ride in mini_bike.iterrows():
    print(ride['Start Station Latitude'])
    break
    

40.78275


In [ ]:
for index, loc in mta_loc.iterrows():
    try:
        station = mta_loc.STATION[index]
        linename = mta_loc.LINENAME[index]
        division = mta_loc.DIVISION[index]

        search_query = "https://maps.googleapis.com/maps/api/geocode/json?address="
        for word in station.split(" "):
            search_query += word + '+'

        search_query += 'station+' + linename + '+new+york+'
        search_query += division + '&key=' + secret_key.google_api_key
#         print(search_query)
        response = requests.get(search_query)
        result = json.loads(response.text)

        mta_loc.loc[[index], 'lat'] = result['results'][0]['geometry']['location']['lat']
        mta_loc.loc[[index], 'long'] = result['results'][0]['geometry']['location']['lng']
    except:
        print(mta_loc.STATION[index])
    time.sleep(1)